In [ ]:
pip install langchain

In [ ]:
pip install -qU langchain-openai

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = ""

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo",openai_api_base="xxx")

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="你好")])

response.content

In [ ]:
# 没有 history memory

from langchain_core.messages import HumanMessage

question_1 = "你好，我叫小明"
result_1 = model.invoke([HumanMessage(content=f"{question_1}")])
print(result_1.content)

question_2 = "我叫什么名字"
result_2 = model.invoke([HumanMessage(content=f"{question_2}")])
print(result_2.content)

In [ ]:
# 简单实现 history memory

from langchain_core.messages import HumanMessage

store = []

question_1 = "你好，我叫小明"
result_1 = model.invoke([HumanMessage(content=f"{question_1}")])
print(result_1.content)

store.append({"human": question_1, "ai": result_1.content})

question_2 = "我叫什么名字"
result_2 = model.invoke([HumanMessage(content=f"history: {store} {question_2} ")])
print(result_2.content)

In [ ]:
pip install langchain_community

In [ ]:
# 使用官方模块实现

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {"configurable": {"session_id": "abc2"}}

question_1 = "你好，我叫小明"
result_1 = with_message_history.invoke([HumanMessage(content=f"{question_1}")],config=config)
print(result_1.content)

question_2 = "我叫什么名字"
result_2 = with_message_history.invoke([HumanMessage(content=f"{question_2}")],config=config)
print(result_2.content)

In [ ]:
# 加入prompt

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "在每一句话结尾都加上 回复完毕",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc2"}}

question_1 = "你好，我叫小明"
result_1 = with_message_history.invoke([HumanMessage(content=f"{question_1}")],config=config)
print(result_1.content)

question_2 = "我叫什么名字"
result_2 = with_message_history.invoke([HumanMessage(content=f"{question_2}")],config=config)
print(result_2.content)

In [ ]:
# 减少历史条数

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "在每一句话结尾都加上 回复完毕",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc2"}}

question_1 = "你好，我叫小明"
result_1 = with_message_history.invoke({"messages":[HumanMessage(content=f"{question_1}")]},config=config)
print(result_1.content)

question_t = "今天天气不错"
result_t = with_message_history.invoke({"messages":[HumanMessage(content=f"{question_t}")]},config=config)
print(result_t.content)

question_t = "今天天气不错"
result_t = with_message_history.invoke({"messages":[HumanMessage(content=f"{question_t}")]},config=config)
print(result_t.content)

question_2 = "我叫什么名字"
result_2 = with_message_history.invoke({"messages":[HumanMessage(content=f"{question_2}")]},config=config)
print(result_2.content)